
## YOLO V2, YOLO 9000
- YOLO V2와 9000은 같은 Paper에 실림
- YOLOv2 : 성능과 속도를 모두 개선 및 SSD(Single Shot MultiBox Detector)를 능가
- YOLO 9000 : 9,000 개의 오브젝트 카테고리를 검출하는데 성공
- 목표 : 분류 정확도는 유지하면서 recall과 localization 에 대해 증가시키겠다

- YOLOv2는 네트워크의 크기를 조절하여 FPS(Frames Per Second)와 MaP(Mean Average Precision)을 균형 있게 조절 가능

![yolov2_graph](img/yolov2_graph.png)

### 성능향상의 요인
- (1) Batch Normalization 사용
    - 모든 컨볼루션 레이어에 배치 정규화 추가
    - 정규화는 학습시 빠르게 수렴 및 정규화(Regularization) 효과 존재.
    
![yolo_v2_bn](img/yolo_v2_bn.png)

- mini batch단위로 특징맵 각 채널의 평균과 표준편차를 구한다. 
- 정규화(normalize)시 평균이 0이 되며, 값의 범위는 대부분 -1에서 1사이가 된다.
- 0으로 나누어질경우를 방지하기위해 분모에 epslion(ε)을 더한다.
- 정규화된 특징맵에 채널별로 감마(γ)와 베타(β)를 더해준다
- 배치 정규화 파라미터 감마와 베타는 채널 수만큼 존재한다.

<hr>

- (2) High Resolution Classifier (고해상도 분류기)
    - 학습과정은 2단계로 구성
        - 2단계로 나누는 이유는 처음부터 경계 박스와 클래스 분류를 같이 학습시키기가 어렵기 때문
    - 1단계) 학습을 안정시키기 위해서 네트워크의 앞단과 뒷단을 나누고 **앞단을 선행학습**
        - ImageNet 데이터(고해상도 이미지)를 epoch 10 학습
        - Classfication 네트워크는 고해상도 이미지에 잘 동작.  
    - 2단계)**앞쪽의 레이어들이 어느정도 학습된 이후에 뒷쪽에 레이어를 추가 및 전체 학습**
        - **2단계 학습과정에서 앞쪽 레이어들의 가중치는 Fine Tuning됨**
        - 2단계는 네트워크의 입력을 448x448로 받음.

<hr>

- (3) Convolutional
    - YOLO V1 은 FC 2개가 존재 -> YOLO V2는 FC2 삭제 및 Conv 층만 존재
    - conv만 사용하기 때문에 가장자리에 존재하는 셀들의 receptive field 는 영상 전체를 커버 불가
    - 최종 특징맵의 크기를 홀수x홀수(13x13)로 만들기 위한 448x448 -> 416x416으로 변경
        - 추론 속도도 향상 및 최종 grid cell들의 receptive field가 더 커지는 효과 발생
    - 최종 출력 레이어는 13x13이며 그리드셀이 특징맵의 중앙에 위치가능하여 중앙의 오브젝트를 검출시 효과적

<hr>

- (4) Anchor Boxes
    - 경계박스를 처음부터 직접 예측(FC) -> Anchor Box를 초기값으로 사용하여 예측
    - 모든 앵커 박스에 대해 그것이 오브젝트일 확률을 예측하고 클래스를 분류
    - grid cell이 아닌 anchor box로 예측하면 accuracy 성능이 조금 줄어들긴 하는데, recall율이 많이 늘어났기 때문에 안정적으로 됨.
    
<hr>

- (5) new network
    - Darknet-19를 특징 추출기로 사용 / VGG-19와 유사하지만 연산량이 훨씬 적다
    - Darknet은 Maxpool이 굉장히 빠르게 시작되며, 중간 중간에 1x1 컨볼루션을 통해서 특징맵의 채널 수를 절반으로 줄임
    - YOLO는 GoogleNet 따라감, YOLO V2는 3x3 필터가 많아서 VGG-19와 유사한 Darknet 선택

![vgg_vs_darknet-19](img/vgg_vs_darknet-19.png)

<hr>

- (6) Dimenion Clusters
    - 기존의 앵커박스는 종횡비가 1, 1/2, 2인 직사각형을 사용
    - 클러스터링을 할 때 유클리디안 거리를 이용하는 표준적인 k-means를 사용 (박스끼리 합치는 것)
        - k-means 사용시 아무박스나 가져다붙이니 경계 박스의 크기에 비례해서 k-means의 에러가 커진다
        - **여기선 K-Means에러를 줄이는 것이 아닌, IOU 점수를 높이기 위한 목적으로 좋은 앵커 박스를 선택하는 것이 중요하다 **
    - 클러스터링 갯수 k를 늘릴 수록 클러스터링 결과와 라벨과의 IOU가 커져 Recall이 상승
        - k 가 커지면 모델의 복잡도가 상승하기에 k=5
        
    - **즉 클러스터링을 통해 선택한 경계박스 5개의 평균 IOU가 앵커박스 9개를 선택했을때보다 높다.**
    
![YOLOV2_kmeans](img/YOLOV2_kmeans.png)

![YOLOV2_kmeans_graph](img/YOLOV2_kmeans_graph.png)


<hr>

- (7) Direct Location Prediction
    - 앵커 박스를 이용하면 학습 초기단계에서 모델이 불안정해지는 문제를 해결
        - 학습 불안정의 주요 원인은 학습 초기에 박스의 (x,y) 위치가 너무 랜덤하게 예측되기 때문
        - **초기값을 랜덤으로 하게 될 경우 어느 정도 정확한 오프셋을 예측하기까지 오랜 시간이 걸림**
    - IOU를 고려한 이용한 확률값을 고려해서 Offset 범위에 대한) 그 후 BoxRegression방법을 썻더니 좋았다고 함

![yolov2_Direct_Location_Prediction](img/yolov2_Direct_Location_Prediction.PNG)

<hr>

- (8) passthrough
    - YOLO는 하나의 특징맵에서만 경계 박스 후보를 제안한다
    - **YOLO v2의 13x13 특징맵은 큰 오브젝트 검출할 때는 크기가 충분하지만, 작은 오브젝트에 대해서는 불충분 할 수 있다.**
    - **skip-layer를 사용하여 해당문제를 해결**
        - 26x26크기의 중간 특징맵을 skip하여 13x13레이어에 붙임(concatenate)
        - 26x26특징맵에는 13x13 특징맵에 비해 고해상도의 특징이 들어있음)
        - 두 개의 특징맵의 크기가 다르기 때문에 그냥 붙일 수는 없고, 26x26x512를 13x13x(512*4) 특징맵으로 변환하여 붙임
        - 26x26의 특징맵의 한 포인트에서 주위 2x2의 데이터를 4채널로 모양을 바꿔주는 것.
        
![yolov2_passthrough](img/yolov2_passthrough.PNG)

<hr>

<hr>

- (9) Multi-Scale Training
    - **YoloV2는 FCNN(Fully Convolutional Neural Network)이기 때문에 입력 이미지의 해상도를 그때 그때 변경 할 수 있다. (FC 일경우는 안됨)**
    - 네트워크의 입력 영상의 크기는 다양한 해상도에 대해 골고루 학습을 하기위해 10번의 배치마다 (320, 352, ... 576, 608) 크기로 resize 되어 입력된다.
 
     - 입력 이미지가 커지면 최종 grid의 cell 수가 많아진다
         - **해당 yolo v2 네트워크는 이미지를 1/32로 줄이기 때문에 입력 이미지의 해상도가 32의 배수이면됨.**
         - (320, 320+32, 320+32*2, 320+32*3 .... 320+32*9=608)
    - 당연히 Input이미지의 크기가 클수록 좋은 결과를 보임
    
<hr>
    
- (10) Fine Grained Features
    - 최종 특징 레이어 그리드셀이 기존 YOLO 7x7에서 13x13으로 커짐
    - 그리드셀은 2개의 경계박스를 제안 -> 5개의 경계박스 후보
    - 경계박스 안에 있는 오브젝트에 대한 클래스 확률 공유 -> 경계박스의 클래스 확률을 독립적으로 제안

![YOLO-V1_VS_YOLO_V2](img/YOLO-V1_VS_YOLO_V2.png)




### Training for classification vs detection

- Training for Classification
    - Imagenet 1000 classes for 160 epochs
    - 다양한 Data Augmentation 진행
    - Initial training : 224x224 -> 448x448 로 진행, Fine-tuning 시켜서 좋았다.

- Training for Detection
    - 3x3 filter를 적용
    - VOC(Detection용 데이터) 최종적으로 5 box predict예측 20classes를 예측

## Joint Classification and Detection

### YOLO 9000 - Stronger
- Directed 그래프로 나타낼 수 있는 WordNet 그래프가 있음 (not a tree)
- 워드트리로 분류할 때 주어진 관련어의 각 하의어의 확률에 대하여 모든 노드에서 조건부 확률을 구한다.

- COCO (Detection Dataset) Imagenet의 탑 9,000개의 클래스 데이터셋(Classification)을 조합해서 굉장히 큰 스케일의 디텍터를 학습시키려고 함

![coco_imagenet_wordtree](img/coco_imagenet_wordtree.PNG)

- 검출 작업에서 이미지넷은 코코와 겨우 44개의 오브젝트 카테고리를 공유
- YOLO9000은 대부분 분류 데이터에 의해 학습되었음.

- 경우1) Detection Image
    - 네트워크가 Detection 용 이미지를 받을 경우에는 Detection 이미지 그대로 그냥 로스를 역전파 진행

- 경우2) Classification Image
    - 분류용 로스에 대해서는, 해당 클래스와 그것의 **상위 레벨**에 대해서만 역전파 진행 (하위라벨은 뭔지 모르니까)
    - 분류용 이미지가 들어오면 분류 로스만 역전파
        - 이렇게 하면 경계 박스를 찾을 때 높은 확률로 클래스를 예측 가능.
        - 경계 박스도 실제 위치(ground truth)와 적어도 .3 IOU 이상일 경우에만 objectness loss를 역전파 진행.


### 실험결과
- COCO 데이터셋의 동물 데이터에 대해서는 잘 일반화하여 학습했기 때문에 이미지넷에만 나타나는 새로운 동물들은 학습하기 쉬움.
- 그러나 COCO 에서는 의류나 장비와 같은 데이터에 대한 경계 박스가 없기 때문에 성능이 나쁨

![coco_imagenet_wordtree_result_image](img/coco_imagenet_wordtree_result_image.png)

![coco_imagenet_result](img/coco_imagenet_result.PNG)

### 결론
- YOLOv2는 최고 성능의 빠른 검출 시스템으로, 이미지 사이즈에 따라 속도와 정확도간의 트레이드오프를 컨트롤 할 수 있음 (YOLO V1에 비해 많이 성장)
- YOLO9000은 9,000개의 오브젝트 카테고리 검출을 위한 실시간 프레임워크로 검출과 분류에 모두 최적화되었음.
    - 이미지넷과 코코의 데이터셋을 합쳐서 워드트리를 구성하여 9,000장의 오브젝트를 학습시키는 방법을 소개
    - **계층적 분류를 이용해 다양한 데이터셋을 조합시키는 방법은 분류나 세그멘테이션 문제에서도 유용할 것**

## 참고문헌
- https://blog.naver.com/sogangori/221011203855
